In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import xml.etree.ElementTree as et
from io import BytesIO
from zipfile import ZipFile
from bs4 import BeautifulSoup as bs
import xmltodict
import json
import dart_fss as dart
from lxml import html
from xml.etree.ElementTree import parse
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import numpy as np
import datetime
try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize
import OpenDartReader
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
kosdaq = pd.read_excel("C:/Users/SDA08/Desktop/workspace/bobae_/0_프로젝트/0_ESG.bb/2_preprocessing.bb/DART_kosdaq_preprocessing.bb/kosdaq_report.xlsx")

In [3]:
len(kosdaq.columns)

30

In [4]:
kosdaq.columns

Index(['고유번호', '종목코드', '회사명', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역',
       '사업자등록번호', '주소', '업종코드', '설립일', '최대주주_성명', '최대주주_관계', '최대주주_주식종류',
       '최대주주_기초주식수', '최대주주_기초주식지분율', '최대주주_기말주식수', '최대주주_기말주식지분율', '소액주주수',
       '주주수', '소액주주비율', '소액주주_주식수', '총주식수', '소액주주_주식지분율', '감사의견', '감사의견_강조사항',
       '감사의견_핵심감사사항'],
      dtype='object')

In [8]:
company_info = kosdaq.iloc[:,:30]
company_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   고유번호          1718 non-null   int64  
 1   종목코드          1718 non-null   int64  
 2   회사명           1718 non-null   object 
 3   업종            1718 non-null   object 
 4   주요제품          1715 non-null   object 
 5   상장일           1718 non-null   int64  
 6   결산월           1718 non-null   object 
 7   대표자명          1718 non-null   object 
 8   홈페이지          1572 non-null   object 
 9   지역            1718 non-null   object 
 10  사업자등록번호       690 non-null    object 
 11  주소            690 non-null    object 
 12  업종코드          690 non-null    float64
 13  설립일           690 non-null    float64
 14  최대주주_성명       658 non-null    object 
 15  최대주주_관계       658 non-null    object 
 16  최대주주_주식종류     658 non-null    object 
 17  최대주주_기초주식수    658 non-null    object 
 18  최대주주_기초주식지분율  658 non-null  

In [11]:
company_info.drop(['고유번호', '종목코드', '주요제품', '상장일', '결산월', '홈페이지', '지역', '사업자등록번호', '주소', '업종코드', '설립일'], axis=1, inplace=True)
company_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   회사명           1718 non-null   object 
 1   업종            1718 non-null   object 
 2   대표자명          1718 non-null   object 
 3   최대주주_성명       658 non-null    object 
 4   최대주주_관계       658 non-null    object 
 5   최대주주_주식종류     658 non-null    object 
 6   최대주주_기초주식수    658 non-null    object 
 7   최대주주_기초주식지분율  658 non-null    float64
 8   최대주주_기말주식수    658 non-null    object 
 9   최대주주_기말주식지분율  658 non-null    float64
 10  소액주주수         1704 non-null   object 
 11  주주수           1704 non-null   object 
 12  소액주주비율        1704 non-null   object 
 13  소액주주_주식수      1704 non-null   object 
 14  총주식수          1704 non-null   object 
 15  소액주주_주식지분율    1704 non-null   object 
 16  감사의견          1704 non-null   object 
 17  감사의견_강조사항     1704 non-null   object 
 18  감사의견_핵심감사사항   1704 non-null 

In [12]:
company_info.to_excel('company_info.xlsx', index=False)

In [14]:
kosdaq['감사의견'] = kosdaq['감사의견'].str.replace('\n', '')

소액주주 지분율(%) 실수화

In [20]:
import pandas as pd

# 쉼표 제거 및 NaN 값 처리
숫자열 = ['소액주주수', '주주수', '소액주주_주식수', '총주식수']
kosdaq[숫자열] = kosdaq[숫자열].replace({',': '', '-': pd.NA}, regex=True)

# 빈 문자열이면 0으로 대체
kosdaq[숫자열] = kosdaq[숫자열].fillna(0)

# 정수형으로 변환
kosdaq[숫자열] = kosdaq[숫자열].astype('int64')

# 소숫점이 있는 열은 실수형으로 변환
kosdaq[['소액주주비율', '소액주주_주식지분율']] = kosdaq[['소액주주비율', '소액주주_주식지분율']].replace('-', pd.NA)

# NA 값을 평균값으로 대체 (문자열을 제외하고 계산)
mean_value = kosdaq[['소액주주비율', '소액주주_주식지분율']].apply(pd.to_numeric, errors='coerce').mean()
kosdaq[['소액주주비율', '소액주주_주식지분율']] = kosdaq[['소액주주비율', '소액주주_주식지분율']].fillna(mean_value)

# 정수형으로 변환
kosdaq[['소액주주비율', '소액주주_주식지분율']] = kosdaq[['소액주주비율', '소액주주_주식지분율']].astype(float)


In [25]:
result_info = kosdaq.drop(['고유번호', '종목코드', '주요제품', '상장일', '결산월', '홈페이지', '지역', '사업자등록번호', '주소', '업종코드', '설립일'], axis=1, inplace=True)

In [27]:
result_info= kosdaq.iloc[:,:26]
result_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   회사명           1718 non-null   object 
 1   업종            1718 non-null   object 
 2   대표자명          1718 non-null   object 
 3   최대주주_성명       658 non-null    object 
 4   최대주주_관계       658 non-null    object 
 5   최대주주_주식종류     658 non-null    object 
 6   최대주주_기초주식수    658 non-null    object 
 7   최대주주_기초주식지분율  658 non-null    float64
 8   최대주주_기말주식수    658 non-null    object 
 9   최대주주_기말주식지분율  658 non-null    float64
 10  소액주주수         1718 non-null   int64  
 11  주주수           1718 non-null   int64  
 12  소액주주비율        1718 non-null   float64
 13  소액주주_주식수      1718 non-null   int64  
 14  총주식수          1718 non-null   int64  
 15  소액주주_주식지분율    1718 non-null   float64
 16  감사의견          1704 non-null   object 
 17  감사의견_강조사항     1704 non-null   object 
 18  감사의견_핵심감사사항   1704 non-null 

In [28]:
result_info.to_excel('Final.xlsx', index=False)

In [3]:
list= pd.read_excel("C:/Users/SDA08/Desktop/workspace/bobae_/0_프로젝트/0_ESG.bb/2_preprocessing.bb/DART_kosdaq_preprocessing.bb/Final.xlsx")

In [5]:
list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   회사명           1718 non-null   object 
 1   업종            1718 non-null   object 
 2   대표자명          1718 non-null   object 
 3   최대주주_성명       658 non-null    object 
 4   최대주주_관계       658 non-null    object 
 5   최대주주_주식종류     658 non-null    object 
 6   최대주주_기초주식수    658 non-null    object 
 7   최대주주_기초주식지분율  658 non-null    float64
 8   최대주주_기말주식수    658 non-null    object 
 9   최대주주_기말주식지분율  658 non-null    float64
 10  소액주주수         1718 non-null   int64  
 11  주주수           1718 non-null   int64  
 12  소액주주비율        1718 non-null   float64
 13  소액주주_주식수      1718 non-null   int64  
 14  총주식수          1718 non-null   int64  
 15  소액주주_주식지분율    1718 non-null   float64
 16  감사의견          1704 non-null   object 
 17  감사의견_강조사항     1704 non-null   object 
 18  감사의견_핵심감사사항   1704 non-null 

In [23]:
# '가산점' column 추가 및 초기값 설정
list['가산점'] = 2

# '최대주주_기말주식지분율'이 60.00 이상인 경우 '가산점' 열에서 -1을 차감하고, 해당 값이 없는 경우에는 -0.5를 차감
list.loc[list['최대주주_기말주식지분율'] >= 60.00, '가산점'] -= 1
list.loc[list['최대주주_기말주식지분율'].isnull(), '가산점'] -= 0.5

# '감사의견' 열에 '적정'이라는 값이 들어가 있지 않거나 값이 없는 경우 '가산점' 열에서 -1을 차감
list.loc[(list['감사의견'] != '적정') | (list['감사의견'].isnull()), '가산점'] -= 1

In [42]:
list.to_excel('list(1).xlsx', index=False)